In [2]:
import pandas as pd
import geopandas as gpd

from bokeh.io import output_notebook

In [3]:
from bokeh.plotting import figure, show
from bokeh.models import (
    LinearAxis,
    Range1d,
    Tabs,
    TabPanel,
    ColumnDataSource,
    HoverTool,
)


def get_multi_tab_line_plot(
    df1, df2, loop_list, category_column, conflict_measure, ntl_measure
):
    tabs = []

    for ts in loop_list:
        # # Create a new Bokeh figure
        p = figure(
            title=f"Comparing Nighttime Light Trends and {conflict_measure.capitalize()}",
            x_axis_label="Month",
            width=800,
            height=400,
            x_axis_type="datetime",
        )

        p.y_range.start = df2[conflict_measure].min()
        p.y_range.end = df2[conflict_measure].max()

        p.extra_y_ranges = {
            "y2": Range1d(
                start=df1[df1[category_column] == ts][ntl_measure].min(),
                end=df1[df1[category_column] == ts][ntl_measure].max(),
            )
        }
        # Add the second line plot
        ntl_source = ColumnDataSource(df1[df1[category_column] == ts])
        ntl_line = p.vbar(
            "date",
            top=ntl_measure,
            source=ntl_source,
            legend_label=ntl_measure,
            width=2000 * 2000 * 750,
            color="blue",
            alpha=0.7,
            y_range_name="y2",
        )

        # Add the first line plot
        conflict_source = ColumnDataSource(df2[df2[category_column] == ts])
        conflict_line = p.line(
            "date",
            conflict_measure,
            source=conflict_source,
            legend_label=f"{conflict_measure.capitalize()}",
            line_width=2,
            color="red",
            alpha=0.7,
        )

        # Customize legend
        p.legend.location = "top_left"
        p.legend.click_policy = "hide"  # Optional: Allow clicking to hide/show lines

        p.add_layout(LinearAxis(y_range_name="y2", axis_label="Luminosity"), "right")

        # Add hover tool for conflict data
        hover_conflict = HoverTool(
            renderers=[conflict_line],  # Reference the renderer directly
            tooltips=[
                ("Date", "@date{%F}"),
                (f"{conflict_measure.capitalize()}", f"@{conflict_measure}"),
            ],
            formatters={
                "@date": "datetime",  # Format the date
            },
            mode="vline",
        )

        # Add hover tool for NTL data
        hover_ntl = HoverTool(
            renderers=[ntl_line],  # Reference the renderer directly
            tooltips=[
                ("Date", "@date{%F}"),
                (ntl_measure, f"@{ntl_measure}"),
            ],
            formatters={
                "@date": "datetime",  # Format the date
            },
            mode="vline",
        )

        p.add_tools(hover_conflict, hover_ntl)

        tab = TabPanel(child=p, title=ts)
        tabs.append(tab)

    return tabs

In [4]:
def get_multi_tab_bar_plot(df1, loop_list, category_column, ntl_measure, title):
    tabs = []

    for ts in loop_list:
        # # Create a new Bokeh figure
        p = figure(
            title=f"{title}",
            x_axis_label="",
            width=800,
            height=400,
            x_axis_type="datetime",
        )

        ntl_source = ColumnDataSource(df1[df1[category_column] == ts])
        p.y_range.start = df1[ntl_measure].min() - 5
        p.y_range.end = df1[ntl_measure].max() + 5
        # Add the second line plot

        ntl_line = p.vbar(
            x="date",
            top=ntl_measure,
            source=ntl_source,
            legend_label=ntl_measure,
            width=2000 * 2000 * 750,
            color="#4E79A7",
            alpha=0.7,
        )

        # Customize legend
        p.legend.location = "top_left"
        p.legend.click_policy = "hide"  # Optional: Allow clicking to hide/show lines

        # Add hover tool for NTL data
        hover_ntl = HoverTool(
            renderers=[ntl_line],  # Reference the renderer directly
            tooltips=[
                ("Date", "@date{%F}"),
                (ntl_measure, f"@{ntl_measure}"),
            ],
            formatters={
                "@date": "datetime",  # Format the date
            },
            mode="vline",
        )

        p.add_tools(hover_ntl, hover_ntl)

        tab = TabPanel(child=p, title=ts)
        tabs.append(tab)

    return tabs

# Conflict and Nighttime Lights - Updated

This notebook analyses how Nighttime Lights changed as the conflict in the country changed. Conflict is measured using ACLED data and a conflict index is calculated as geometric mean between Number of fatalities and Number of events. This conflict index is then compred to Nighttime lights (measured as lumiinosity) wihout gas flaring. The Nighttime Lights data is retrieved from [BlackMarble](https://datapartnership.org/myanmar-economic-monitor/notebooks/nighttime-lights/analysis-2023/README.html). 

## Insights

The following questions quide the data analysis - 

- **How is NTL different in EAO controlled areas and other regions?**
For this, we used the control map of Myanmar as of March 2024 released by the [Special Advisory Council](https://specialadvisorycouncil.org/wp-content/uploads/2024/05/SAC-M-Effective-Control-in-Myanmar-2024-Update-ENGLISH.pdf). The contestation and control is divided into 8 regions starting from 'stable junta control' to 'complete resistance control'. We expect to see that the regions that have moved into 'complete resistance control' have also experienced high increase in conflict. 

- **Is there a direct correlation between Conflict Index and Nighttime Lights?**
We did not find a direct correlation between the two variables at a national or township level.

- **How did NTL change in regions that saw high increase in conflict compared to 6 months prior?**
There is no direct correlation between the two variables found yet. 



### Nighttime Lights in EAO Controlled Areas Vs Junta Controlled Areas

In [5]:
mmr_adm3 = gpd.read_file("../../data/boundaries/mmr_polbnda_adm3_250k_mimu.shp")

In [6]:
conflict_monthly = pd.read_stata(
    "../../data/conflict/ACLED_01_Jan_2021_21_Oct_2024_township_monthly_spatial.dta"
)
conflict_monthly.rename(columns={"month": "date"}, inplace=True)
conflict_6_monthly = pd.read_stata(
    "../../data/conflict/ACLED_Apr_2022_Sep_2024_township_spatial_period_change.dta"
)

In [7]:
ntl_adm3_monthly = pd.read_csv("../../data/ntl/ntl_mmr_adm3_monthly.csv")
ntl_adm3_monthly = mmr_adm3[["ST", "DT", "TS", "geometry"]].merge(
    ntl_adm3_monthly, on=["ST", "DT", "TS"]
)
ntl_adm3_monthly["date"] = pd.to_datetime(ntl_adm3_monthly["date"])

In [8]:
ntl_adm3_annual = pd.read_csv("../../data/ntl/ntl_mmr_adm3_annual.csv")
ntl_adm3_annual = mmr_adm3[["ST", "DT", "TS", "geometry"]].merge(
    ntl_adm3_annual, on=["ST", "DT", "TS"]
)
ntl_adm3_annual["date"] = pd.to_datetime(ntl_adm3_annual["date"], format="%Y")

In [9]:
def get_conflict_groups(code):
    if code <= 2:
        return "SEC Control"
    if code > 2 and code <= 6:
        return "Contested Areas"
    if code > 6:
        return "EAO Control"

In [10]:
control_categories = pd.read_csv("../../data/conflict/control_category_map.csv")
control_categories.rename(columns={"Township": "TS"}, inplace=True)
control_categories["category"] = control_categories["code_no"].apply(
    lambda x: get_conflict_groups(x)
)

eao_controlled = list(
    control_categories[control_categories["category"] == "EAO Control"]["TS"].unique()
)
junta_controlled = list(
    control_categories[control_categories["category"] == "SEC Control"]["TS"].unique()
)
contested = list(
    control_categories[control_categories["category"] == "Contested Areas"][
        "TS"
    ].unique()
)

In [11]:
ntl_adm3_monthly = ntl_adm3_monthly.merge(control_categories, on="TS")
ntl_adm3_annual = ntl_adm3_annual.merge(control_categories, on="TS")

In [12]:
ntl_adm3_annual_grouped = (
    ntl_adm3_monthly.groupby([pd.Grouper(key="date", freq="YS"), "category"])[
        ["ntl_nogf_5km_sum", "ntl_sum"]
    ]
    .mean()
    .reset_index()
    .sort_values(by=["date"])
)
ntl_adm3_monthly_grouped = (
    ntl_adm3_monthly.groupby([pd.Grouper(key="date", freq="MS"), "category"])[
        ["ntl_nogf_5km_sum", "ntl_sum"]
    ]
    .mean()
    .reset_index()
    .sort_values(by=["date"])
)

In [13]:
control_code_dict = {
    1: "Stable junta control",
    2: "Junta dependent on local proxy militias for control",
    3: "Junta forces under regular attack from resistance forces; administration functions remain weak",
    4: "Resistance controls growing territory but still cannot consolidate fuller control",
    5: "Limited junta movement, dependent on ceasefires",
    6: "Junta control receding; resistance defending increasing territories & asserting local administration",
    7: "Strong resistance control & local administration—90%+ of township",
    8: "Full resistance control & local administration—whole township",
}

control_category_list = ["SEC Control", "EAO Control", "Contested Areas"]

In [14]:
conflict_monthly = conflict_monthly.merge(control_categories, on="TS")
# conflict_monthly['code_no'] = conflict_monthly['code_no'].astype(str)

In [15]:
# Import necessary libraries
output_notebook()

df1 = ntl_adm3_annual_grouped
# df1=df1[df1['date']>'2021-01-1']
ntl_measure = "ntl_nogf_5km_sum"


tabs = get_multi_tab_bar_plot(
    df1,
    loop_list=control_category_list,
    category_column="category",
    ntl_measure=ntl_measure,
    title="Nighttime Lights Trend",
)

# Create the Tabs layout
tabs_layout = Tabs(tabs=tabs)

# Show the tabs
show(tabs_layout)

# # Show the plot
# show(p)

Loading BokehJS ...

**There is a huge peak in the currently EAO controlled regions back in April 2023 where there is massive peak in nighttime lights. The same peak is not visible in the junta controlled regions.** There is a lot more consistent light in regions where junta forces are under regular attack (category 3).

In [16]:
ntl_annual_baseline = (
    ntl_adm3_annual_grouped[ntl_adm3_annual_grouped["date"] == "2019-01-01"]
    .groupby(["category"])["ntl_nogf_5km_sum"]
    .first()
)

ntl_adm3_annual_grouped = ntl_adm3_annual_grouped.set_index("category")
ntl_adm3_annual_grouped["baseline"] = ntl_adm3_annual_grouped.index.map(
    ntl_annual_baseline
)

ntl_adm3_annual_grouped["pchange_from_2019"] = (
    100
    * (
        ntl_adm3_annual_grouped["ntl_nogf_5km_sum"]
        - ntl_adm3_annual_grouped["baseline"]
    )
    / ntl_adm3_annual_grouped["baseline"]
)

In [17]:
# Import necessary libraries
output_notebook()
from bokeh.models import Tabs

df1 = ntl_adm3_annual_grouped.reset_index()
# df1=df1[df1['date']>'2021-01-1']
ntl_measure = "pchange_from_2019"


tabs = get_multi_tab_bar_plot(
    df1,
    loop_list=control_category_list,
    category_column="category",
    ntl_measure=ntl_measure,
    title="Percentage change in NTL from 2019",
)

# Create the Tabs layout
tabs_layout = Tabs(tabs=tabs)

# Show the tabs
show(tabs_layout)

# # Show the plot
# show(p)

Loading BokehJS ...

In [18]:
ntl_adm3_annual_grouped = ntl_adm3_annual_grouped.sort_values(by=["category", "date"])
ntl_adm3_annual_grouped["pchange_from_py"] = (
    ntl_adm3_annual_grouped.groupby(["category"])["ntl_nogf_5km_sum"].pct_change() * 100
)

In [19]:
# Import necessary libraries
output_notebook()
from bokeh.models import Tabs

df1 = ntl_adm3_annual_grouped.reset_index()
# df1=df1[df1['date']>'2021-01-1']
ntl_measure = "pchange_from_py"


tabs = get_multi_tab_bar_plot(
    df1,
    loop_list=control_category_list,
    category_column="category",
    ntl_measure=ntl_measure,
    title="Percentage change in NTL from previous year",
)

# Create the Tabs layout
tabs_layout = Tabs(tabs=tabs)

# Show the tabs
show(tabs_layout)

# # Show the plot
# show(p)

Loading BokehJS ...

In [20]:
ntl_adm3_monthly_grouped["year"] = ntl_adm3_monthly_grouped["date"].dt.year
ntl_adm3_monthly_grouped["month"] = ntl_adm3_monthly_grouped["date"].dt.month

ntl_monthly_baseline = (
    ntl_adm3_monthly_grouped[
        (ntl_adm3_monthly_grouped["date"] >= "2017-01-01")
        & (ntl_adm3_monthly_grouped["date"] <= "2019-12-01")
    ]
    .groupby(["category", "month"])["ntl_nogf_5km_sum"]
    .mean()
)

ntl_adm3_monthly_grouped = ntl_adm3_monthly_grouped.set_index(["category", "month"])
ntl_adm3_monthly_grouped["baseline"] = ntl_adm3_monthly_grouped.index.map(
    ntl_monthly_baseline
)

ntl_adm3_monthly_grouped["pchange_from_2017_2019"] = (
    100
    * (
        ntl_adm3_monthly_grouped["ntl_nogf_5km_sum"]
        - ntl_adm3_monthly_grouped["baseline"]
    )
    / ntl_adm3_monthly_grouped["baseline"]
)

In [21]:
# Import necessary libraries
output_notebook()
from bokeh.models import Tabs

df1 = ntl_adm3_monthly_grouped.reset_index()
# df1=df1[df1['date']>'2021-01-1']
ntl_measure = "pchange_from_2017_2019"


tabs = get_multi_tab_bar_plot(
    df1[df1["date"] >= "2017-01-01"],
    loop_list=control_category_list,
    category_column="category",
    ntl_measure=ntl_measure,
    title="Percentage change in NTL from corresponding months in 2017-2019",
)

# Create the Tabs layout
tabs_layout = Tabs(tabs=tabs)

# Show the tabs
show(tabs_layout)

# # Show the plot
# show(p)

Loading BokehJS ...

In [22]:
ntl_adm3_monthly_grouped = ntl_adm3_monthly_grouped.sort_values(
    by=["category", "month", "year"]
)
ntl_adm3_monthly_grouped["py_same_month"] = ntl_adm3_monthly_grouped.groupby(
    ["category", "month"]
)["ntl_nogf_5km_sum"].shift(1)
ntl_adm3_monthly_grouped["pchange_from_py_same_month"] = (
    100
    * (
        ntl_adm3_monthly_grouped["ntl_nogf_5km_sum"]
        - ntl_adm3_monthly_grouped["py_same_month"]
    )
    / ntl_adm3_monthly_grouped["py_same_month"]
)

In [23]:
# Import necessary libraries
output_notebook()
from bokeh.models import Tabs

df1 = ntl_adm3_monthly_grouped.reset_index()
# df1=df1[df1['date']>'2021-01-1']
ntl_measure = "pchange_from_py_same_month"


tabs = get_multi_tab_bar_plot(
    df1,
    loop_list=control_category_list,
    category_column="category",
    ntl_measure=ntl_measure,
    title="Percentage change in NTL from corresponding months in 2017-2019",
)

# Create the Tabs layout
tabs_layout = Tabs(tabs=tabs)

# Show the tabs
show(tabs_layout)

# # Show the plot
# show(p)

Loading BokehJS ...

In [24]:
very_severe_conflict = list(
    conflict_monthly[
        (conflict_monthly["conflict_category"] == "Very severe conflict")
        & (conflict_monthly["date"] >= "2024-03-01")
    ]["TS"].unique()
)
severe_conflict = list(
    conflict_monthly[
        (conflict_monthly["conflict_category"] == "Severe conflict")
        & (conflict_monthly["date"] >= "2024-03-01")
    ]["TS"].unique()
)
no_conflict = list(
    conflict_monthly[
        (conflict_monthly["conflict_category"] == "No conflict")
        & (conflict_monthly["date"] >= "2024-03-01")
    ]["TS"].unique()
)
mild_conflict = list(
    conflict_monthly[
        (conflict_monthly["conflict_category"] == "Mild conflict")
        & (conflict_monthly["date"] >= "2024-03-01")
    ]["TS"].unique()
)
moderate_conflict = list(
    conflict_monthly[
        (conflict_monthly["conflict_category"] == "Moderate conflict")
        & (conflict_monthly["date"] >= "2024-03-01")
    ]["TS"].unique()
)

In [25]:
def get_current_conflict_category(x):
    if x in very_severe_conflict:
        return "very_severe_conflict"
    elif x in severe_conflict:
        return "severe_conflict"
    elif x in no_conflict:
        return "no_conflict"
    elif x in mild_conflict:
        return "mild_conflict"
    elif x in "moderate_conflict":
        return "moderate_conflict"

In [26]:
ntl_adm3_monthly["conflict_category"] = ntl_adm3_monthly["TS"].apply(
    lambda x: get_current_conflict_category(x)
)

In [27]:
ntl_adm3_annual_grouped_conflict = (
    ntl_adm3_monthly.groupby([pd.Grouper(key="date", freq="YS"), "conflict_category"])[
        ["ntl_nogf_5km_sum", "ntl_sum"]
    ]
    .mean()
    .reset_index()
    .sort_values(by=["date"])
)
ntl_adm3_monthly_grouped_conflict = (
    ntl_adm3_monthly.groupby([pd.Grouper(key="date", freq="MS"), "conflict_category"])[
        ["ntl_nogf_5km_sum", "ntl_sum"]
    ]
    .mean()
    .reset_index()
    .sort_values(by=["date"])
)

In [28]:
ntl_annual_baseline = (
    ntl_adm3_annual_grouped_conflict[
        ntl_adm3_annual_grouped_conflict["date"] == "2019-01-01"
    ]
    .groupby(["conflict_category"])["ntl_nogf_5km_sum"]
    .first()
)

ntl_adm3_annual_grouped_conflict = ntl_adm3_annual_grouped_conflict.set_index(
    "conflict_category"
)
ntl_adm3_annual_grouped_conflict["baseline"] = (
    ntl_adm3_annual_grouped_conflict.index.map(ntl_annual_baseline)
)

ntl_adm3_annual_grouped_conflict["pchange_from_2019"] = (
    100
    * (
        ntl_adm3_annual_grouped_conflict["ntl_nogf_5km_sum"]
        - ntl_adm3_annual_grouped_conflict["baseline"]
    )
    / ntl_adm3_annual_grouped_conflict["baseline"]
)

In [29]:
conflict_category_list = [
    "no_conflict",
    "mild_conflict",
    "moderate_conflict",
    "severe_conflict",
    "very_severe_conflict",
]

In [30]:
# Import necessary libraries
output_notebook()
from bokeh.models import Tabs

df1 = ntl_adm3_annual_grouped_conflict.reset_index()
# df1=df1[df1['date']>'2021-01-1']
ntl_measure = "pchange_from_2019"


tabs = get_multi_tab_bar_plot(
    df1[df1["date"] >= "2019-01-01"],
    loop_list=conflict_category_list,
    category_column="conflict_category",
    ntl_measure=ntl_measure,
    title="Percentage change in NTL from 2019",
)

# Create the Tabs layout
tabs_layout = Tabs(tabs=tabs)

# Show the tabs
show(tabs_layout)

# # Show the plot
# show(p)

Loading BokehJS ...

In [31]:
ntl_adm3_annual_grouped_conflict = ntl_adm3_annual_grouped_conflict.sort_values(
    by=["conflict_category", "date"]
)
ntl_adm3_annual_grouped_conflict["pchange_from_py"] = (
    ntl_adm3_annual_grouped_conflict.groupby(["conflict_category"])[
        "ntl_nogf_5km_sum"
    ].pct_change()
    * 100
)

In [32]:
# Import necessary libraries
output_notebook()
from bokeh.models import Tabs

df1 = ntl_adm3_annual_grouped_conflict.reset_index()
# df1=df1[df1['date']>'2021-01-1']
ntl_measure = "pchange_from_py"


tabs = get_multi_tab_bar_plot(
    df1,
    loop_list=conflict_category_list,
    category_column="conflict_category",
    ntl_measure=ntl_measure,
    title="Percentage change in NTL from previous year",
)

# Create the Tabs layout
tabs_layout = Tabs(tabs=tabs)

# Show the tabs
show(tabs_layout)

# # Show the plot
# show(p)

Loading BokehJS ...

In [33]:
ntl_adm3_monthly_grouped_conflict["year"] = ntl_adm3_monthly_grouped_conflict[
    "date"
].dt.year
ntl_adm3_monthly_grouped_conflict["month"] = ntl_adm3_monthly_grouped_conflict[
    "date"
].dt.month

ntl_monthly_baseline = (
    ntl_adm3_monthly_grouped_conflict[
        (ntl_adm3_monthly_grouped_conflict["date"] >= "2017-01-01")
        & (ntl_adm3_monthly_grouped_conflict["date"] <= "2019-12-01")
    ]
    .groupby(["conflict_category", "month"])["ntl_nogf_5km_sum"]
    .mean()
)

ntl_adm3_monthly_grouped_conflict = ntl_adm3_monthly_grouped_conflict.set_index(
    ["conflict_category", "month"]
)
ntl_adm3_monthly_grouped_conflict["baseline"] = (
    ntl_adm3_monthly_grouped_conflict.index.map(ntl_monthly_baseline)
)

ntl_adm3_monthly_grouped_conflict["pchange_from_2017_2019"] = (
    100
    * (
        ntl_adm3_monthly_grouped_conflict["ntl_nogf_5km_sum"]
        - ntl_adm3_monthly_grouped_conflict["baseline"]
    )
    / ntl_adm3_monthly_grouped_conflict["baseline"]
)

In [34]:
# Import necessary libraries
output_notebook()
from bokeh.models import Tabs

df1 = ntl_adm3_monthly_grouped_conflict.reset_index()
# df1=df1[df1['date']>'2021-01-1']
ntl_measure = "pchange_from_2017_2019"


tabs = get_multi_tab_bar_plot(
    df1[df1["date"] > "2017-01-01"],
    loop_list=conflict_category_list,
    category_column="conflict_category",
    ntl_measure=ntl_measure,
    title="Percentage change in NTL from corresponding months in 2017-2019",
)

# Create the Tabs layout
tabs_layout = Tabs(tabs=tabs)

# Show the tabs
show(tabs_layout)

# # Show the plot
# show(p)

Loading BokehJS ...

In [35]:
ntl_adm3_monthly_grouped_conflict = ntl_adm3_monthly_grouped_conflict.sort_values(
    by=["conflict_category", "month", "year"]
)
ntl_adm3_monthly_grouped_conflict["py_same_month"] = (
    ntl_adm3_monthly_grouped_conflict.groupby(["conflict_category", "month"])[
        "ntl_nogf_5km_sum"
    ].shift(1)
)
ntl_adm3_monthly_grouped_conflict["pchange_from_py_same_month"] = (
    100
    * (
        ntl_adm3_monthly_grouped_conflict["ntl_nogf_5km_sum"]
        - ntl_adm3_monthly_grouped_conflict["py_same_month"]
    )
    / ntl_adm3_monthly_grouped_conflict["py_same_month"]
)

In [37]:
# Import necessary libraries
output_notebook()
from bokeh.models import Tabs

df1 = ntl_adm3_monthly_grouped_conflict.reset_index()
# df1=df1[df1['date']>'2021-01-1']
ntl_measure = "pchange_from_py_same_month"


tabs = get_multi_tab_bar_plot(
    df1,
    loop_list=conflict_category_list,
    category_column="conflict_category",
    ntl_measure=ntl_measure,
    title="Percentage change in NTL from corresponding months in previous year",
)

# Create the Tabs layout
tabs_layout = Tabs(tabs=tabs)

# Show the tabs
show(tabs_layout)

# # Show the plot
# show(p)

Loading BokehJS ...